In [1]:
file1=r"C:\Users\swadhin biswal\Desktop\happy data scientist\python\PROJECTS\project1\Consumer_Complaints_train.csv"
file2=r"C:\Users\swadhin biswal\Desktop\happy data scientist\python\PROJECTS\project1\Consumer_Complaints_test_share.csv"

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


In [3]:
ld_train=pd.read_csv(file1)
ld_test=pd.read_csv(file2)

In [4]:
ld_train.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [5]:
ld_test["Consumer disputed?"]=np.nan


In [6]:
ld_test.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Complaint ID', 'Consumer disputed?'],
      dtype='object')

In [7]:
ld_test["data"]="test"
ld_train["data"]="train"
ld_test=ld_test[ld_train.columns]

In [8]:
ld=pd.concat([ld_train,ld_test],axis=0)

In [9]:
ld["Complaint ID"].nunique()

598027

In [10]:
ld["ZIP code"].nunique()

27342

In [11]:
ld.drop(["Complaint ID","ZIP code"],axis=1,inplace=True)

In [12]:
ld.shape

(598027, 17)

In [13]:
ld.dtypes

Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Date sent to company            object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
data                            object
dtype: object

In [14]:
ld.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,train


In [15]:
for col in ld.columns:
    print(col,ld[col].nunique())

Date received 1759
Product 12
Sub-product 47
Issue 95
Sub-issue 68
Consumer complaint narrative 92118
Company public response 10
Company 3437
State 62
Tags 3
Consumer consent provided? 4
Submitted via 6
Date sent to company 1708
Company response to consumer 7
Timely response? 2
Consumer disputed? 2
data 2


In [16]:
for col in ['Date received','Date sent to company']:
    ld[col]=pd.to_datetime(ld[col],infer_datetime_format=True)

In [17]:
ld.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,train


In [18]:
ld['day_diff']=pd.to_numeric(ld['Date sent to company']-ld['Date received'])

In [19]:
ld.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,data,day_diff
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,train,86400000000000
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,train,518400000000000
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,train,1814400000000000
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,train,0
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,train,691200000000000


In [20]:
ld.drop(["Date sent to company","Date received"],inplace=True,axis=1)

In [21]:
ld.columns

Index(['Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'Tags', 'Consumer consent provided?', 'Submitted via',
       'Company response to consumer', 'Timely response?',
       'Consumer disputed?', 'data', 'day_diff'],
      dtype='object')

In [22]:
ld.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
data                                 0
day_diff                             0
dtype: int64

In [23]:
ld.shape

(598027, 16)

In [24]:
ld.nunique()

Product                            12
Sub-product                        47
Issue                              95
Sub-issue                          68
Consumer complaint narrative    92118
Company public response            10
Company                          3437
State                              62
Tags                                3
Consumer consent provided?          4
Submitted via                       6
Company response to consumer        7
Timely response?                    2
Consumer disputed?                  2
data                                2
day_diff                          420
dtype: int64

In [25]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    ld[varname]=np.where(pd.isnull(ld[col]),1,0)
    ld.drop([col],1,inplace=True)

In [26]:
ld.head()

,Product,Issue,Company,State,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,data,day_diff,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
0,Credit card,Billing statement,Wells Fargo & Company,MI,Web,Closed with explanation,Yes,No,train,86400000000000,1,1,1,1,0,1
1,Bank account or service,"Making/receiving payments, sending money",Santander Bank US,PA,Referral,Closed,Yes,No,train,518400000000000,0,1,1,1,1,1
2,Credit reporting,Incorrect information on credit report,Equifax,CA,Referral,Closed with non-monetary relief,Yes,No,train,1814400000000000,1,0,1,1,1,1
3,Credit card,Billing statement,U.S. Bancorp,GA,Web,Closed with monetary relief,Yes,No,train,0,1,1,0,0,0,0
4,Credit card,Transaction issue,Bank of America,MA,Web,Closed with explanation,Yes,No,train,691200000000000,1,1,1,1,1,1


In [27]:
ld_test.isnull().sum()

Date received                        0
Product                              0
Sub-product                      34752
Issue                                0
Sub-issue                        73060
Consumer complaint narrative    101049
Company public response          96830
Company                              0
State                              925
ZIP code                           926
Tags                            102735
Consumer consent provided?       85742
Submitted via                        1
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
dtype: int64

In [28]:
ld.drop(['Company'],inplace=True,axis=1)

In [29]:
ld.head()

,Product,Issue,State,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,data,day_diff,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
0,Credit card,Billing statement,MI,Web,Closed with explanation,Yes,No,train,86400000000000,1,1,1,1,0,1
1,Bank account or service,"Making/receiving payments, sending money",PA,Referral,Closed,Yes,No,train,518400000000000,0,1,1,1,1,1
2,Credit reporting,Incorrect information on credit report,CA,Referral,Closed with non-monetary relief,Yes,No,train,1814400000000000,1,0,1,1,1,1
3,Credit card,Billing statement,GA,Web,Closed with monetary relief,Yes,No,train,0,1,1,0,0,0,0
4,Credit card,Transaction issue,MA,Web,Closed with explanation,Yes,No,train,691200000000000,1,1,1,1,1,1


In [30]:
k=ld['Issue'].value_counts()
for val in k.index[0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    ld[varname]=np.where(ld['Issue']==val,1,0)
del ld['Issue']

In [31]:
ld.head()

,Product,State,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,data,day_diff,Sub_product_isNan,Sub_issue_isNan,...,Issue_Loan_modification_collection_foreclosure,Issue_Incorrect_information_on_credit_report,Issue_Loan_servicing__payments__escrow_account,Issue_Cont'd_attempts_collect_debt_not_owed,Issue_Account_opening__closing__or_management,Issue_Disclosure_verification_of_debt,Issue_Communication_tactics,Issue_Deposits_and_withdrawals,Issue_Application__originator__mortgage_broker,Issue_Billing_disputes
0,Credit card,MI,Web,Closed with explanation,Yes,No,train,86400000000000,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Bank account or service,PA,Referral,Closed,Yes,No,train,518400000000000,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Credit reporting,CA,Referral,Closed with non-monetary relief,Yes,No,train,1814400000000000,1,0,...,0,1,0,0,0,0,0,0,0,0
3,Credit card,GA,Web,Closed with monetary relief,Yes,No,train,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Credit card,MA,Web,Closed with explanation,Yes,No,train,691200000000000,1,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
k=ld['State'].value_counts()
for val in k.index[0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    ld[varname]=np.where(ld['State']==val,1,0)
del ld['State']

In [33]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(ld[col],prefix=col,drop_first=True)
    ld=pd.concat([temp,ld],1)
    ld.drop([col],1,inplace=True)

In [34]:
ld.head()

,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,...,State_NJ,State_PA,State_IL,State_VA,State_MD,State_OH,State_NC,State_MI,State_AZ,State_WA
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
train=ld.loc[ld["data"]=="train",]
test=ld.loc[ld["data"]=="test",]
test.drop(["data","Consumer disputed?"],axis=1,inplace=True)


In [37]:
train.drop(["data"],axis=1,inplace=True)
train["Consumer disputed?"]=np.where(train["Consumer disputed?"]=="Yes",1,0)

C:\Users\swadhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
train["Consumer disputed?"].value_counts()

0    376990
1    101431
Name: Consumer disputed?, dtype: int64

In [39]:
train.shape

(478421, 56)

In [40]:
test.shape

(119606, 55)

In [41]:
lr1=LogisticRegression()
lr2=LogisticRegression()

In [42]:
train_train,train_val=train_test_split(train,test_size=0.2,random_state=2)

In [43]:
x_train_train=train_train.drop(["Consumer disputed?"],axis=1)
y_train_train=train_train["Consumer disputed?"]
x_train_val=train_val.drop(["Consumer disputed?"],axis=1)

In [44]:
x_train_train.shape

(382736, 55)

In [45]:
lr1.fit(x_train_train,y_train_train)

C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [46]:

p=lr1.predict_proba(x_train_val)[:,1]
roc_auc_score(train_val["Consumer disputed?"],p)

0.5287059355830243

In [47]:
lr2.fit(x_train_train,y_train_train)
p=lr2.predict_proba(x_train_val)[:,1]
roc_auc_score(train_val["Consumer disputed?"],p)

C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5287059355830243

In [ ]:
rf=RandomForestClassifier(n_estimators=500,max_depth=15,min_samples_split=5)
params={"criterion":["gini","entropy"]}
gs_rf=GridSearchCV(rf,param_grid=params,cv=5,scoring="roc_auc")
gs_rf.fit(x_train_train,y_train_train)
p=gs_rf.predict_proba(x_train_val)[:,1]
roc_auc_score(train_val["Consumer disputed?"],p)

In [106]:
gs_rf.best_params_

{'min_samples_leaf': 6}

In [ ]:
0.6220236384397624

In [50]:
dtree=DecisionTreeClassifier()
dtree.fit(x_train_train,y_train_train)
p=dtree.predict_proba(x_train_val)[:,1]
roc_auc_score(train_val["Consumer disputed?"],p)

0.572223846442545

In [58]:
et=DecisionTreeClassifier()
et.fit(x_train_train,y_train_train)
p=et.predict(x_train_val)
roc_auc_score(train_val["Consumer disputed?"],p)

IndexError: too many indices for array

In [59]:
p=et.predict(x_train_val)

In [61]:
roc_auc_score(train_val["Consumer disputed?"],p)

0.5059726621151958

In [52]:
from sklearn.ensemble import GradientBoostingClassifier

In [53]:
gb=GradientBoostingClassifier()
gb.fit(x_train_train,y_train_train)
p=gb.predict_proba(x_train_val)[:,1]
roc_auc_score(train_val["Consumer disputed?"],p)

0.61473977340317

In [96]:
xg=XGBClassifier(objective="binary:logistic",learning_rate=0.4,max_depth=4)
params={}
gs=GridSearchCV(xg,param_grid=params,cv=5,scoring="roc_auc")
gs.fit(x_train_train,y_train_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.4,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_child_weight': [1, 2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [97]:
p=gs.predict_proba(x_train_val)[:,1]  
roc_auc_score(train_val["Consumer disputed?"],p)

0.620917104643879

In [ ]:
0.6210559787373799

In [91]:
gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.4,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [66]:
train.columns

Index(['Timely response?_Yes',
       'Company response to consumer_Closed with explanation',
       'Company response to consumer_Closed with monetary relief',
       'Company response to consumer_Closed with non-monetary relief',
       'Company response to consumer_Closed with relief',
       'Company response to consumer_Closed without relief',
       'Company response to consumer_Untimely response', 'Submitted via_Fax',
       'Submitted via_Phone', 'Submitted via_Postal mail',
       'Submitted via_Referral', 'Submitted via_Web', 'Product_Consumer Loan',
       'Product_Credit card', 'Product_Credit reporting',
       'Product_Debt collection', 'Product_Money transfers',
       'Product_Mortgage', 'Product_Other financial service',
       'Product_Payday loan', 'Product_Prepaid card', 'Product_Student loan',
       'Product_Virtual currency', 'Consumer disputed?', 'day_diff',
       'Sub_product_isNan', 'Sub_issue_isNan',
       'Consumer_complaint_narrative_isNan', 'Company_publ

In [67]:
x_train=train.drop(["Consumer disputed?"],axis=1)
y_train=train["Consumer disputed?"]


In [76]:

xg=XGBClassifier(objective="binary:logistic",max_depth=2)
xg.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [78]:
real=y_train
f_beta=[]
for cutoff in cutoffs:
    predicted=(p_xg1>cutoff).astype(int)
    
    tp=((predicted==1) & (real==1)).sum()
    tn=((predicted==0) & (real==0)).sum()
    fp=((predicted==1) & (real==0)).sum()
    fn=((predicted==0) & (real==1)).sum()
    
    p=tp+fn
    n=tn+fp
    precision=tp/(tp+fp)
    recall=tp/p
    f_score=((1+4)*precision*recall)/((4*precision)+recall)
    
    f_beta.append(f_score)

C:\Users\swadhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in longlong_scalars
  del sys.path[0]


In [77]:
cutoffs=np.linspace(0.01,0.99,99)
p_xg1=xg.predict_proba(x_train)[:,1]

In [79]:
cutoffs[f_beta==max(f_beta)]

array([0.13])

In [80]:
new=(p_xg1>0.13).astype(int)

In [81]:
roc_auc_score(y_train,new)

0.547944761532786

# nn=MLPClassifier()
nn.fit(x_train_train,y_train_train)
p=nn.predict_proba(x_train_val)[:,1]
roc_auc_score(train_val["Consumer disputed?"],p)

KeyboardInterrupt: 